<a href="https://colab.research.google.com/github/amun83/Koscom-FinancialDataCamp/blob/master/FD102_Forecasting_Natural_Gas_ETN_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Sources
* https://www.investing.com/commodities/natural-gas
* https://www.eia.gov/naturalgas/data.php
* http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/ao_index.html

### ETP
* 신한 천연가스 선물 ETN(H)
* 신한 레버리지 천연가스 선물 ETN(H)
* 신한 인버스 천연가스 선물 ETN(H)
* 신한 인버스 2X 천연가스 선물 ETN(H)
* 삼성 레버리지 천연가스 선물 ETN
* 삼성 인버스 2X 천연가스 선물 ETN

In [0]:
!pip install -q html5lib
!pip install -q lxml
!pip install -q pandas_datareader
!pip install -q finance-datareader
!pip install -q gspread

In [0]:
import requests
import datetime
import urllib.parse
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup
from pandas.plotting import scatter_matrix

%matplotlib inline
import matplotlib.pyplot as plt

import pandas_datareader as pdr
import FinanceDataReader as fdr

from IPython.display import display

## 데이터 읽기

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [29]:
# Dow Jones Commodity Index Natural Gas TR
with open("/gdrive/My Drive/Natural Gas Data/DJCI.csv", "r") as f:
  df_DJCI = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_DJCI.head(3), df_DJCI.tail(3)]))

,Dow Jones Commodity Index Natural Gas TR,Dow Jones Commodity Index Natural Gas ER,Dow Jones Commodity Index Natural Gas
Effective date,,,
2018-11-16,0.85,0.60,233.33
2018-11-15,0.80,0.56,219.85
2018-11-14,0.97,0.68,266.34
2008-11-04,21.64,15.70,392.55
2008-11-03,20.50,14.87,371.83
2008-10-31,20.33,14.75,368.84


In [30]:
# Henry Hub Natural Gas Spot
with open("/gdrive/My Drive/Natural Gas Data/NG_Spot.csv", "r") as f:
  df_Spot = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_Spot.head(3), df_Spot.tail(3)]))
  

,Henry Hub Natural Gas Spot Price (Dollars per Million Btu)
Date,
1997-01-07,3.82
1997-01-08,3.80
1997-01-09,3.61
2018-11-09,3.74
2018-11-12,3.96
2018-11-13,4.10


In [31]:
# NYMEX Natural Gas Futures
with open("/gdrive/My Drive/Natural Gas Data/NG_Futures.csv", "r") as f:
  df_Futures = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_Futures.head(3), df_Futures.tail(3)]))


,Natural Gas Futures Contract 1 (Dollars per Million Btu),Natural Gas Futures Contract 2 (Dollars per Million Btu),Natural Gas Futures Contract 3 (Dollars per Million Btu),Natural Gas Futures Contract 4 (Dollars per Million Btu)
Date,,,,
1993-12-20,NaN,NaN,NaN,1.894
1993-12-21,NaN,NaN,NaN,1.830
1993-12-22,NaN,NaN,NaN,1.859
2018-11-12,3.788,3.800,3.660,3.293
2018-11-13,4.101,4.147,4.019,3.712
2018-11-14,4.837,4.898,4.770,4.472


In [36]:
# EIA Weekly Natural Gas Storage Report
with open("/gdrive/My Drive/Natural Gas Data/NG_Storage.csv", "r") as f:
  df_Storage = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_Storage.head(3), df_Storage.tail(3)]))
  

,Source,East Region,Midwest Region,Mountain Region,Pacific Region,South Central Region,Salt,NonSalt,Total Lower 48
Week ending,,,,,,,,,
2010-01-01,Derived EIA Weekly Estimates,769,900,195,268,985,159,826,"3,117"
2010-01-08,Derived EIA Weekly Estimates,703,820,185,257,886,123,763,"2,850"
2010-01-15,Derived EIA Weekly Estimates,642,750,176,246,793,91,702,"2,607"
2018-10-26,EIA-912,826,956,180,262,919,234,686,"3,143"
2018-11-02,EIA-912,831,980,182,265,949,253,696,"3,208"
2018-11-09,EIA-912,835,991,181,266,974,272,702,"3,247"


In [33]:
# NOAA Arctic Oscillation Index
with open("/gdrive/My Drive/Natural Gas Data/AO_Index.csv", "r") as f:
  df_AOI = pd.read_csv(f, header = None, index_col = 0, parse_dates = [[0, 1, 2]])
  display(pd.concat([df_AOI.head(3), df_AOI.tail(3)])) 

,3
0_1_2,
1979-01-01,-1.556
1979-01-02,-0.888
1979-01-03,0.255
2018-10-29,0.508
2018-10-30,0.754
2018-10-31,0.776


In [34]:
# Crude Oil WTI Futures
with open("/gdrive/My Drive/Natural Gas Data/WTI_Futures.csv", "r") as f:
  df_WTI = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_WTI.head(3), df_WTI.tail(3)]))  

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2018-11-19,56.98,57.31,57.55,56.83,-,-0.56%
2018-11-18,57.30,57.00,57.36,56.93,-,1.49%
2018-11-16,56.46,56.58,57.96,55.89,159.00K,0.00%
1990-01-04,23.41,23.90,23.92,22.83,53.07K,-1.14%
1990-01-03,23.68,23.19,23.80,23.00,50.06K,3.45%
1990-01-02,22.89,21.81,22.92,21.79,45.18K,4.90%


In [35]:
# US Dollar Index
with open("/gdrive/My Drive/Natural Gas Data/Dollar_Index.csv", "r") as f:
  df_DI = pd.read_csv(f, index_col = 0, parse_dates = True)
  display(pd.concat([df_DI.head(3), df_DI.tail(3)]))

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2018-11-19,96.36,96.45,96.57,96.32,-,-0.07%
2018-11-18,96.43,96.43,96.45,96.38,-,-0.04%
2018-11-16,96.47,97.05,97.06,96.40,-,-0.47%
1990-01-03,94.42,94.15,94.52,94.08,-,0.14%
1990-01-02,94.29,93.13,94.31,93.08,-,1.16%
1990-01-01,93.21,93.19,93.31,93.08,-,0.00%


In [39]:
'''
polysi = pd.concat([polysi, df_pvinsights, df_sunsirs_ps[["Price_USD_KG", "Price_TDY", "CNY_USD"]]], axis = 1)
polysi.columns = ("PVInsights", "Sunsirs", "Sunsirs_CNY", "CNY_USD")
polysi.head(5)
'''
M = pd.concat([
    df_DJCI['Dow Jones Commodity Index Natural Gas TR'],
    df_Spot,
    df_Futures,
    df_Storage['Total Lower 48'],
    df_AOI,
    df_WTI['Price'],
    df_DI['Price']
], axis = 1)

M.columns = ['DJCI', 'Spot', 'Fut1', 'Fut2', 'Fut3', 'Fut4', 'Storage', 'AOI', 'WTI', 'DI']

M.tail(100)

,DJCI,Spot,Fut1,Fut2,Fut3,Fut4,Storage,AOI,WTI,DI
2018-08-08,0.59,2.99,2.949,2.955,2.989,3.077,NaN,-0.883,66.94,95.05
2018-08-09,0.59,3.02,2.955,2.959,2.992,3.077,NaN,-0.967,66.81,95.50
2018-08-10,0.59,2.96,2.944,2.949,2.982,3.067,"2,386",-0.914,67.63,96.36
2018-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.914,NaN,NaN
2018-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.897,NaN,NaN
2018-08-13,0.59,2.92,2.930,2.936,2.971,3.059,NaN,-1.114,67.20,96.39
2018-08-14,0.60,3.02,2.959,2.966,3.002,3.092,NaN,-1.411,67.04,96.73
2018-08-15,0.59,3.02,2.940,2.948,2.985,3.078,NaN,-1.752,65.01,96.70
2018-08-16,0.58,3.02,2.908,2.913,2.950,3.046,NaN,-1.885,65.46,96.64
2018-08-17,0.59,3.01,2.946,2.949,2.986,3.082,"2,435",-1.752,65.91,96.10
